In [ ]:
from phi.torch.flow import *

In [ ]:
backend.default_backend().get_default_device()

In [ ]:
math.get_precision()

In [ ]:
from phi.torch.flow import *

BOUNDS = Box(x=1, y=1, z=1)
RESOLUTION = 50
DENSITY = CenteredGrid(
    values=0,
    extrapolation=extrapolation.ZERO_GRADIENT,
    bounds=BOUNDS,
    resolution=spatial(x=RESOLUTION, y=RESOLUTION, z=RESOLUTION),
)
VELOCITY = StaggeredGrid(
    values=0,
    extrapolation=extrapolation.ZERO,
    bounds=BOUNDS,
    resolution=spatial(x=RESOLUTION, y=RESOLUTION, z=RESOLUTION),
)
INFLOW = CenteredGrid(
    values=Sphere(center=tensor([0.5, 0.5, 0.5], channel(vector='x,y,z')), radius=0.1),
    extrapolation=extrapolation.ZERO_GRADIENT,
    bounds=BOUNDS,
    resolution=spatial(x=RESOLUTION, y=RESOLUTION, z=RESOLUTION),
)


@jit_compile
def step(v, s, p, dt, inflow_rate, inflow):
    s = advect.mac_cormack(s, v, dt) + inflow_rate * resample(inflow, to=s, soft=True)
    buoyancy = resample(s * (0, 0.1, 0), to=v)
    v = advect.semi_lagrangian(v, v, dt) + buoyancy * dt
    v, p = fluid.make_incompressible(v, (), Solve('CG', 1e-3, x0=p))
    return v, s, p

step(VELOCITY, DENSITY, None, 0.2, 0.2, INFLOW)
step(VELOCITY, DENSITY, None, 0.2, 0.2, INFLOW)

In [ ]:
# np.savez('phiflow.npz', density=DENSITY.numpy(), inflow=INFLOW.numpy())